In [1]:
import numpy as np
import sys

In [2]:
with open('Grokking-Deep-Learning/reviews.txt', encoding='utf-8') as file:
    raw_reviews = list(map(str.strip, file.readlines()))
#     raw_reviews = file.readlines()
    
with open('Grokking-Deep-Learning/labels.txt', encoding='utf-8') as file:
    raw_labels = list(map(str.strip, file.readlines()))

In [3]:
# with open('raw_reviews.pkl', 'wb') as file:
#     pickle.dump(raw_reviews, file)

In [4]:
# tokens = list(map(lambda x: set(x.split(' ')), raw_reviews))

tokens = [set(x.split(' ')) for x in raw_reviews] # список множеств слов из каждой статьи

In [5]:
import pickle
with open('tokens.pkl', 'wb') as file:
    pickle.dump(tokens, file)

In [6]:
# vocab1 = set()
# for sent in tokens:
#     for word in sent:
#         if (len(word) > 0):
#             vocab1.add(word)
# vocab1 = list(vocab1)

vocab = list({word for token in tokens for word in token if len(word) > 0}) # словарь всех слов во всех статьях

In [7]:
# word2index1 = {}
# for i, word in enumerate(vocab):
#     word2index1[word] = i

word2index = {key: value for value, key in enumerate(vocab)} # словарь индексов всех слов

In [8]:
# input_dataset1 = list()
# for sent in tokens:
#     sent_indices = list()
#     for word in sent:
#         try:
#             sent_indices.append(word2index[word])
#         except:
#             ''
#     input_dataset1.append(list(set(sent_indices)))

# перекодировка tokens из слов и индексы

input_dataset = [[word2index[word] for word in token if len(word) > 0] for token in tokens]

In [9]:
target_dataset = [1 if label == 'positive' else 0 for label in raw_labels] # оцифровка target

inputs = np.zeros((len(input_dataset[:-1000]), len(vocab)))
for i in range(inputs.shape[0]):
    for word in input_dataset[:-1000][i]:
        inputs[i][word] = 1

tests = np.zeros((len(input_dataset[-1000:]), len(vocab)))
for i in range(tests.shape[0]):
    for word in input_dataset[-1000:][i]:
        tests[i][word] = 1

targets = np.array(target_dataset[:-1000]).reshape(-1, 1)
outputs = np.zeros((targets.shape[0], 2))
for output, target in zip(outputs, targets):
    output[target] = 1
    
test_targets = np.array(target_dataset[-1000:]).reshape(-1, 1)
test_outputs = np.zeros((test_targets.shape[0], 2))
for test_output, test_target in zip(test_outputs, test_targets):
    test_output[test_target] = 1

In [11]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return (x > 0) * x

def relu2deriv(x):
    return (x > 0)

lr = 0.0001
iterations = 33
hidden_size = 100
batch_size = 100

np.random.seed(1)

weights_0_1 = 0.02 * np.random.random((len(vocab), hidden_size)) - 0.01
weights_1_2 = 0.02 * np.random.random((hidden_size, 2)) - 0.01

func = lambda x: np.sum(x, axis=0)

In [13]:
for iter in range(iterations):
    for i in range(int(len(inputs) / batch_size)):
        batch_start = i * batch_size
        batch_end = min(batch_start + batch_size, len(inputs))
        
        notes = inputs[batch_start: batch_end]
        goal_pred = outputs[batch_start: batch_end]
        
        layer_0 = inputs[batch_start: batch_end]
        layer_1 = relu(layer_0.dot(weights_0_1))
        layer_2 = layer_1.dot(weights_1_2)
        
        layer_2_delta = (layer_2 - goal_pred)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
        
        weights_1_2 -= lr * layer_1.T.dot(layer_2_delta)
        weights_0_1 -= lr * layer_0.T.dot(layer_1_delta)
        
    # Проверка точности на всем массиве тренировоных данных
    layer_0 = inputs
    layer_1 = relu(layer_0.dot(weights_0_1))
    layer_2 = layer_1.dot(weights_1_2)

    error = np.sum((layer_2 - outputs) ** 2)
    correct_train = np.sum(np.array([np.argmax(x) for x in outputs]).reshape(-1, 1) == np.array([np.argmax(x) for x in layer_2]).reshape(-1, 1)) / len(outputs)

    # Проверка точности на тестовых данных
    layer_0 = tests
    layer_1 = relu(layer_0.dot(weights_0_1))
    layer_2 = layer_1.dot(weights_1_2)

    correct_test = np.sum(test_targets == np.array([np.argmax(x) for x in layer_2]).reshape(-1, 1)) / len(test_targets)
    sys.stdout.write(f'I: {iter} Error: {error:.10f} Train-Acc: {correct_train:.5f} Test-Acc: {correct_test:.5f}')
    print()

I: 0 Error: 852.2775829890 Train-Acc: 0.99500 Test-Acc: 0.84600
I: 1 Error: 838.8561814459 Train-Acc: 0.99500 Test-Acc: 0.84700
I: 2 Error: 824.4363008779 Train-Acc: 0.99521 Test-Acc: 0.84600
I: 3 Error: 810.5202046100 Train-Acc: 0.99546 Test-Acc: 0.84600
I: 4 Error: 800.2404583166 Train-Acc: 0.99562 Test-Acc: 0.84600
I: 5 Error: 787.7927482098 Train-Acc: 0.99575 Test-Acc: 0.84600
I: 6 Error: 774.8183469298 Train-Acc: 0.99596 Test-Acc: 0.84600
I: 7 Error: 764.8222515581 Train-Acc: 0.99608 Test-Acc: 0.84600
I: 8 Error: 751.1662305752 Train-Acc: 0.99621 Test-Acc: 0.84600
I: 9 Error: 739.0906594470 Train-Acc: 0.99625 Test-Acc: 0.84600
I: 10 Error: 728.3990546292 Train-Acc: 0.99646 Test-Acc: 0.84600
I: 11 Error: 716.4951013154 Train-Acc: 0.99654 Test-Acc: 0.84700
I: 12 Error: 708.5259363394 Train-Acc: 0.99667 Test-Acc: 0.84700
I: 13 Error: 696.3593940824 Train-Acc: 0.99675 Test-Acc: 0.84700
I: 14 Error: 684.4084253580 Train-Acc: 0.99675 Test-Acc: 0.84500
I: 15 Error: 675.9976717148 Train-A

In [32]:
import pickle

In [34]:
# with open('word2index.pkl', 'wb') as file:
#     pickle.dump(word2index, file)

In [77]:
# with open('weights_0_1.pkl', 'wb') as file:
#     pickle.dump(weights_0_1, file)

In [78]:
# with open('weights_1_2.pkl', 'wb') as file:
#     pickle.dump(weights_1_2, file)

In [12]:
with open('weights_0_1.pkl', 'rb') as file:
    weights_0_1 = pickle.load(file)

In [13]:
with open('weights_1_2.pkl', 'rb') as file:
    weights_1_2 = pickle.load(file)

In [75]:
layer_0 = inputs
layer_1 = relu(layer_0.dot(weights_0_1))
layer_2 = layer_1.dot(weights_1_2)

In [76]:
correct = np.sum(np.array([np.argmax(x) for x in outputs]).reshape(-1, 1) == np.array([np.argmax(x) for x in layer_2]).reshape(-1, 1)) / len(outputs)
correct

0.9735416666666666

In [14]:
word2index['king']

9112

In [16]:
word2index['man']

22951

In [15]:
word2index['queen']

61604

In [17]:
word2index['woman']

44399

In [20]:
pred = weights_0_1[9112] - weights_0_1[22951] + weights_0_1[44399]

In [22]:
queen = weights_0_1[61604]

In [26]:
np.sum((pred - queen))

0.06784773860284869